In [ ]:
%%capture
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from delfi.simulator.Ricker import Ricker


In [ ]:
n_params = 3
seed = 49

T, burn_in = 50, 10
e = np.zeros(T+burn_in)
log_r0 = 3.

thetao = np.array([3.8, 0.3, 10.])
m = Ricker(dim=n_params, 
           burnIn=burn_in, T=T, 
           eps=e, log_r0=log_r0, 
           seed=seed)
p = dd.Uniform(lower=np.array([2.5, 0., 6.0]), 
               upper=np.array([5.0, 1., 14.0]),
               seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)

theta, x = g.gen(1)

#plt.figure(figsize=(16,16))
#plt.plot(x.T)
#plt.show()

print(x)

In [ ]:
from ctypes import CDLL, c_double, c_int, RTLD_GLOBAL


#load the shared object file
ricker = CDLL('/home/mackelab/Desktop/Projects/Biophysicality/code/delfi_mn/delfi/simulator/ricker.so')
n      = (c_double * T)(*np.zeros(T))

ricker.ricker(
    n,                                          # *n   
    (c_double * theta.size)(*theta.flatten()),  # *theta
    (c_double * (T+burn_in))(*e),               # *e
    (c_int * 1)(burn_in),                       # *burn_in
    (c_int * 1)(T),                             # *n_t
    (c_int * 1)(1),                             # *n_reps
    (c_double * 1)(log_r0))                     # *n0
n = np.array(n)

n

In [ ]:

path = '/home/mackelab/Desktop/Projects/Biophysicality/code/delfi_mn/delfi/summarystats/'
acf_c  = CDLL(path + 'acf.so')
nlar_c = CDLL(path + 'nlar.so', mode=RTLD_GLOBAL)

def afc(x, max_lag=10):
    """
    `x' is a matrix containing replicate simulations in its columns.
    sl.acf turns these into acf's    
    """
    assert x.shape[1] == 1 # never tested otherwise. Do not remove without testing

    vafc = np.zeros((max_lag+1, x.shape[1]))
    vafc = (c_double * vafc.size)(*vafc.flatten())
    NAcode = -1e70
    
    #oo <- .C("slacf",acf=as.double(acf),x=as.double(x),as.integer(nrow(x)),as.integer(ncol(x)),
    #        as.integer(max.lag),as.double(NAcode),correlation=as.integer(0),PACKAGE="sl")
    afc = acf_c.slacf(
        vafc,                                               # *afc
        (c_double * (x.shape[0]*x.shape[1]))(*x.flatten()), # *x
        (c_int    * 1)(x.shape[0]),                         # *n
        (c_int    * 1)(x.shape[1]),                         # *n_reps
        (c_int    * 1)(max_lag),                            # *max_lag,
        (c_double * 1)(NAcode),                             # *NAcode
        (c_int    * 1)(0))                                  # correlation
    
    return np.array(vafc)

def nlar(x, lag, power):
    """ 
    relatively efficient polynomial autoregression for multiple reps.
    each column of `x' is a replicate. 
    `lag[i]' is the lag for term i on rhs of autoregression
    `power[i]' is the power for term i on rhs of autoregression 
    """    
    assert x.shape[1] == 1 # never tested otherwise. Do not remove without testing
    
    beta = np.zeros((len(lag), x.shape[1]))
    beta = (c_double * beta.size)(*beta.flatten())
    NAcode = -1e70
    
    #oo <- .C("slnlar",beta = as.double(beta), x = as.double(x),
    #        n=as.integer(nrow(x)),n.reps=as.integer(ncol(x)),n.terms=as.integer(length(lag)),
    #        as.integer(lag),as.integer(power),as.double(NAcode),PACKAGE="sl")    
    nlar_c.slnlar(
        beta,                                               # *beta
        (c_double * (x.shape[0]*x.shape[1]))(*x.flatten()), # *x
        (c_int    * 1)(x.shape[0]),                         # *n
        (c_int    * 1)(x.shape[1]),                         # *n_reps
        (c_int    * 1)(len(lag)),                           # *n_terms
        (c_int    * len(lag))(*lag),                        # *lag
        (c_int    * len(power))(*power),                    # *power
        (c_double * 1)(NAcode)                              # *NAcode
        )
    
    return np.array(beta)

def order_dist(x, z, n_p=3, diff=1):
    """
    Routine to obtain coefficients summarizing distribution of (differenced) columns
    of x, by regression of sorted differenced columns of x on sorted differenced z's. 
    regression is with order `np' polynomial (no intercept as all centred). `diff'
    is order of differencing to apply.
    """
    assert x.shape[1] == 1 # never tested otherwise. Do not remove without testing
    
    beta = np.zeros((np, x.shape[0]))
    beta = (c_double * beta.size)(*beta.flatten())
    
    #oo <- .C("order_reg",beta=as.double(beta), as.double(x),as.double(z),n=as.integer(nrow(x)),
    #        as.integer(ncol(x)),as.integer(np),as.integer(diff),PACKAGE="sl")    
    nlar_c.order_reg(
        beta,                                               # *beta
        (c_double * (x.shape[0]*x.shape[1]))(*x.flatten()), # *x
        (c_double * (z.shape[0]*z.shape[1]))(*z.flatten()), # *z
        (c_int    * 1)(x.shape[0]),                         # *n
        (c_int    * 1)(x.shape[1]),                         # *n_reps
        (c_int    * 1)(n_p),                                # *n_p
        (c_int    * 1)(diff)                                # *diff
        )
    
    return np.array(beta)
    
def summary_stats(Y,y,max_lag=5, lag=[1,1], power=[1,2], n_p=3, diff=1):
    
    acf_Y = afc(Y,max_lag=max_lag)
    acf_y = acf(y,max_lag=max_lag)

    b0_Y = nlar(Y**.3,lag=lag,power=power)
    b0_y = nlar(y**.3,lag=lag,power=power)

    b1_Y = order_dist(Y,y,n_p=n_p,diff=diff)
    b1_y = order_dist(y,y,n_p=n_p,diff=diff)       
    

In [ ]:
np.__config__.show()

In [ ]:
afc(x.T, max_lag=10)

In [ ]:
np.cov(x[0,:-4],x[0,4:])

In [ ]:
"""
  acf.Y <- sl.acf(Y,max.lag=5)
  acf.y <- sl.acf(y,max.lag=5)

  b0.Y <- nlar(Y^.3,lag=c(1,1),power=c(1,2))
  b0.y <- nlar(y^.3,lag=c(1,1),power=c(1,2))

  b1.Y <- order.dist(Y,y,np=3,diff=1)
  b1.y <- order.dist(y,y,np=3,diff=1)   
  
"""